# Лабораторная работа №2: Проведение исследований с логистической и линейной регрессией

## 1. Выбор начальных условий

Был проведен в ЛР №1.

In [ ]:
!pip install kagglehub scikit-learn numpy pandas matplotlib seaborn

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Скачивание Date Fruit Dataset
date_fruit_path = kagglehub.dataset_download("muratkokludataset/date-fruit-datasets")

# Чтение Excel-файла
date_fruit_data = pd.read_excel(f"{date_fruit_path}/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx")

In [ ]:
# Скачивание Concrete Compressive Strength
concrete_strength_path = kagglehub.dataset_download("niteshyadav3103/concrete-compressive-strength")

# Чтение CSV-файла
concrete_data = pd.read_csv(f"{concrete_strength_path}/Concrete Compressive Strength.csv")

100%|██████████| 14.1k/14.1k [00:00<00:00, 6.06MB/s]

Extracting files...


## 2. Создание бейзлайна и оценка качества

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, root_mean_squared_error, r2_score, make_scorer
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline

Разделим датасет для классификации на обучающую и тестовую выборки

In [ ]:
# Разделение на признаки и целевую переменную
X_class = date_fruit_data.drop(columns=['Class'])
y_class = date_fruit_data['Class']

# Разделение на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Преобразование целевой переменной
label_encoder = LabelEncoder()
y_train_class = label_encoder.fit_transform(y_train_class)
y_test_class = label_encoder.transform(y_test_class)

Аналогично разделим датасет для регрессии

In [ ]:
# Разделение на признаки и целевую переменную
X_reg = concrete_data.drop(columns=['Concrete compressive strength '])
y_reg = concrete_data['Concrete compressive strength ']

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Обучим модели для классификации и регрессии из Sklearn и оценим их. Для логистической регрессии нам требуется обязательный препроцессинг данных с помощью StandardScaler.

In [ ]:
scaler = StandardScaler()
X_train_class = scaler.fit_transform(X_train_class)
X_test_class = scaler.transform(X_test_class)

logistic = LogisticRegression(max_iter=10000)
logistic.fit(X_train_class, y_train_class)

y_pred_class = logistic.predict(X_test_class)

accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.9222
F1-Score: 0.9208


In [ ]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train_reg, y_train_reg)

y_pred_reg = linear_regressor.predict(X_test_reg)

rmse = root_mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

RMSE: 9.7967
R²: 0.6275


Итак, точность для встроенной в Sklearn модели логистической регрессии получилась отличной (92.7% точности). Для модели линейной регрессии среднеквадратичная ошибка составила 9.79, что несколько хуже бейзлайна для KNN-регрессора. Попробуем улучшить бейзлайн.

## 3. Улучшение бейзлайна

Для улучшения бейзлайна для логистичекой регрессии будем подбирать гиперпараметр C с помощью GridSearchCV. Для линейной регрессии добавим регуляризацию через Ridge (L2).

In [ ]:
# Пайплайн с нормализацией и логистической регрессией
pipeline_logistic = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('logistic', LogisticRegression(max_iter=10000))
])

# Параметры для подбора
param_grid_logistic = {
    'logistic__C': [0.01, 0.1, 1, 10, 75, 1000],
    'logistic__penalty': ['l2'],  # L2-регуляризация
    'logistic__solver': ['lbfgs']  # Подходит для L2-регуляризации
}

# Подбор гиперпараметров
grid_search_logistic = GridSearchCV(
    pipeline_logistic,
    param_grid_logistic,
    cv=5,
    scoring='accuracy',
    verbose=1
)

grid_search_logistic.fit(X_train_class, y_train_class)

# Лучшие параметры и результаты
best_params_logistic = grid_search_logistic.best_params_
best_score_logistic = grid_search_logistic.best_score_
print(f"Лучшие параметры для Logistic Regression: {best_params_logistic}")
print(f"Лучший F1-Score на кросс-валидации: {best_score_logistic:.4f}")

# Оценка на тестовой выборке
y_pred_class = grid_search_logistic.best_estimator_.predict(X_test_class)
accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1-Score: {f1:.4f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Лучшие параметры для Logistic Regression: {'logistic__C': 1, 'logistic__penalty': 'l2', 'logistic__solver': 'lbfgs'}
Лучший F1-Score на кросс-валидации: 0.9150
Test Accuracy: 0.9278
Test F1-Score: 0.9263


In [ ]:
# Параметры для подбора
param_grid_ridge = {
    'alpha': [0.01, 0.1, 1, 10, 100, 1000]  # Параметр регуляризации
}

grid_search_ridge = GridSearchCV(
    Ridge(),
    param_grid_ridge,
    cv=5,
    scoring='neg_root_mean_squared_error',
    verbose=1
)

grid_search_ridge.fit(X_train_reg, y_train_reg)

# Лучшие параметры и результаты
best_params_ridge = grid_search_ridge.best_params_
best_score_ridge = -grid_search_ridge.best_score_
print(f"Лучшие параметры для Ridge Regression: {best_params_ridge}")
print(f"Лучший RMSE на кросс-валидации: {best_score_ridge:.4f}")

# Оценка на тестовой выборке
ridge_regressor = grid_search_ridge.best_estimator_
y_pred_reg = ridge_regressor.predict(X_test_reg)
rmse = root_mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"Test RMSE: {rmse:.4f}")
print(f"Test R²: {r2:.4f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Лучшие параметры для Ridge Regression: {'alpha': 1000}
Лучший RMSE на кросс-валидации: 10.6589
Test RMSE: 9.7951
Test R²: 0.6277


Итак, результаты немного улучшились по сравнению с бейзлайном.

## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию Logistic and Linear regression, затем обучим модели на тестовых данных и сравним по качеству с реализациями из Sklearn.

In [ ]:
sigmoid_function = lambda z: 1 / (1 + np.exp(-z))

def optimize_gradients(gradient_fn, start_point, learning_rate, max_iter):
    current_point = start_point
    for _ in range(max_iter):
        grad = gradient_fn(current_point)
        current_point -= learning_rate * grad
    return current_point


class CustomLogisticRegression:
    def __init__(self, *, lr=0.01, max_epochs=1000):
        self._learning_rate = lr
        self._max_epochs = max_epochs
        self._parameters = None
        self._X_train = None
        self._y_train = None

    def _compute_gradient(self, params):
        assert self._X_train is not None
        assert self._y_train is not None

        samples, _ = self._X_train.shape
        weights, bias = params[:-1], params[-1]
        predictions = sigmoid_function(np.dot(self._X_train, weights) + bias)

        weight_grad = np.dot(self._X_train.T, (predictions - self._y_train)) / samples
        bias_grad = np.sum(predictions - self._y_train) / samples

        return np.append(weight_grad, bias_grad)

    def fit(self, X, y):
        assert self._parameters is None

        self._X_train = np.array(X)
        self._y_train = np.array(y)
        features = self._X_train.shape[1]

        # Initialize parameters and optimize
        initial_params = np.ones(features + 1) * 0.5
        self._parameters = optimize_gradients(
            self._compute_gradient, initial_params, self._learning_rate, self._max_epochs
        )

    def predict(self, X):
        assert self._parameters is not None

        weights, bias = self._parameters[:-1], self._parameters[-1]
        linear_output = np.dot(X, weights) + bias
        probabilities = sigmoid_function(linear_output)
        return (probabilities >= 0.5).astype(int)

class CustomLinearRegression:
    def __init__(self):
        self._coef_ = None

    def _add_intercept(self, X):
        rows = X.shape[0]
        return np.hstack([np.ones((rows, 1)), X])

    def fit(self, X, y):
        X_intercept = self._add_intercept(X)
        XtX = np.dot(X_intercept.T, X_intercept)
        Xty = np.dot(X_intercept.T, y)
        self._coef_ = np.linalg.solve(XtX, Xty)

    def predict(self, X):
        X_intercept = self._add_intercept(X)
        return np.dot(X_intercept, self._coef_)

Обучим модели и оценим их качество

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score

log_reg = CustomLogisticRegression(lr=0.1, max_epochs=1000)
log_reg.fit(X_train_class, y_train_class)
y_pred_class = log_reg.predict(X_test_class)

accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average="weighted")

print(f"Custom Logistic Regression - Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}")

lin_reg = CustomLinearRegression()
lin_reg.fit(X_train_reg, y_train_reg)
y_pred_reg = lin_reg.predict(X_test_reg)

rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"Custom Linear Regression - RMSE: {rmse:.4f}, R²: {r2:.4f}")

Custom Logistic Regression - Accuracy: 0.1444, F1-Score: 0.0452
Custom Linear Regression - RMSE: 9.7967, R²: 0.6275


Как видно, точность и F1-Score логистической модели сильно упали, а показатели линейной - остались примерно теми же.